In [1]:
!pip -q install ultralytics>=8.3.0 pycocotools opencv-python pandas tqdm

import torch, os
print("PyTorch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
!nvidia-smi -L || echo "No GPU detected (will use CPU)"


PyTorch: 2.2.2
CUDA available: False
GPU 0: NVIDIA A100 80GB PCIe (UUID: GPU-e1c3028b-bfbe-85c5-bc60-3f7c833222d1)


In [3]:
# ==== paths ====
DATA_ROOT = "/mmfs1/home/jacks.local/kkumari/FHB_Project/Wheatproject-v1-coco"  # <— your dataset
OUT_DIR   = "/mmfs1/home/jacks.local/kkumari/FHB_Project/runs_yolo_detection"            # results

# ==== training knobs (shrink if on CPU) ====
IMG_SIZE  = 960          # 640/800/960; larger -> better, slower
BATCH     = 8            # use 2–4 on small GPU; 1–2 on CPU
EPOCHS    = 50
CONF_THR  = 0.25         # inference confidence for counting

os.makedirs(OUT_DIR, exist_ok=True)
assert os.path.exists(DATA_ROOT), f"DATA_ROOT not found: {DATA_ROOT}"
print("DATA_ROOT:", DATA_ROOT)


DATA_ROOT: /mmfs1/home/jacks.local/kkumari/FHB_Project/Wheatproject-v1-coco


In [4]:
import json, shutil
from pathlib import Path

def fix_split(split_dir):
    split_dir = Path(split_dir)
    ann_dir   = split_dir / "annotations"
    img_dir   = split_dir / "images"
    src_json  = ann_dir / "_annotations.coco.json"
    if not src_json.exists():
        print(f"Skip {split_dir.name}: no _annotations.coco.json")
        return None
    dst_json  = ann_dir / "_annotations.coco.fixed.json"
    coco = json.load(open(src_json, "r"))

    kept_images, kept_ids = [], set()
    fixed, dropped = 0, 0

    for im in coco["images"]:
        fname = im.get("file_name","")
        base  = Path(fname).name
        candidates = [
            split_dir/fname,
            img_dir/base,
            split_dir/("images/"+base),
            split_dir/(fname.replace("annotations/","images/"))
        ]
        found = None
        for c in candidates:
            if c.exists():
                found = c; break
        if found is None:
            dropped += 1
            continue
        rel = found.relative_to(split_dir).as_posix()
        if rel != fname: fixed += 1
        im["file_name"] = rel
        kept_images.append(im)
        kept_ids.add(im["id"])

    coco["images"] = kept_images
    coco["annotations"] = [a for a in coco["annotations"] if a["image_id"] in kept_ids]

    bak = ann_dir / "_annotations.coco.orig.json"
    if not bak.exists():
        shutil.copy(src_json, bak)
    json.dump(coco, open(dst_json, "w"), indent=2)
    print(f"[{split_dir.name}] fixed:{fixed}  dropped:{dropped}  kept:{len(kept_images)}  anns:{len(coco['annotations'])}")
    return dst_json

fixed_json = {}
for split in ["train","valid","test"]:
    p = Path(DATA_ROOT)/split
    if p.exists():
        fixed_json[split] = fix_split(p)
fixed_json


[train] fixed:2095  dropped:1  kept:2095  anns:63306
[valid] fixed:164  dropped:0  kept:164  anns:4842
[test] fixed:53  dropped:1  kept:53  anns:1720


{'train': PosixPath('/mmfs1/home/jacks.local/kkumari/FHB_Project/Wheatproject-v1-coco/train/annotations/_annotations.coco.fixed.json'),
 'valid': PosixPath('/mmfs1/home/jacks.local/kkumari/FHB_Project/Wheatproject-v1-coco/valid/annotations/_annotations.coco.fixed.json'),
 'test': PosixPath('/mmfs1/home/jacks.local/kkumari/FHB_Project/Wheatproject-v1-coco/test/annotations/_annotations.coco.fixed.json')}

In [8]:
import json
from pathlib import Path
import numpy as np

# Map label names from your project
NAME2CLS = {
    "healthy": 0, "Healthy": 0,
    "unhealthy": 1, "Unhealthy": 1,
    "diseased": 1, "Diseased": 1
}

def coco_to_yolo_convert(split_dir: Path):
    ann_fixed = split_dir / "annotations/_annotations.coco.fixed.json"
    ann_orig  = split_dir / "annotations/_annotations.coco.json"
    ann_path  = ann_fixed if ann_fixed.exists() else ann_orig
    if not ann_path.exists():
        print(f"[{split_dir.name}] Skipped (no COCO file)")
        return

    coco = json.load(open(ann_path, "r"))
    id2img = {im["id"]: im for im in coco["images"]}
    id2cat = {c["id"]: c["name"] for c in coco["categories"]}

    labels_dir = split_dir / "labels"
    labels_dir.mkdir(parents=True, exist_ok=True)

    # Start fresh
    for f in labels_dir.glob("*.txt"):
        f.unlink()

    lines = {im["id"]: [] for im in coco["images"]}

    for ann in coco["annotations"]:
        img_id = ann["image_id"]
        if img_id not in id2img:
            continue
        w, h = id2img[img_id]["width"], id2img[img_id]["height"]

        cat_name = id2cat.get(ann["category_id"], None)
        cls = NAME2CLS.get(cat_name, None)
        if cls is None:
            continue

        # COCO bbox = [x_min, y_min, width, height]
        x, y, bw, bh = ann["bbox"]
        cx = (x + bw / 2) / w
        cy = (y + bh / 2) / h
        nw = bw / w
        nh = bh / h

        # clip to [0,1]
        cx, cy = np.clip(cx, 0, 1), np.clip(cy, 0, 1)
        nw, nh = np.clip(nw, 0, 1), np.clip(nh, 0, 1)

        if nw <= 0 or nh <= 0:
            continue

        lines[img_id].append(f"{cls} {cx:.6f} {cy:.6f} {nw:.6f} {nh:.6f}")

    written, empty = 0, 0
    for im in coco["images"]:
        stem = Path(im["file_name"]).stem
        lbl_path = labels_dir / f"{stem}.txt"
        arr = lines.get(im["id"], [])
        if len(arr) == 0:
            empty += 1
        else:
            lbl_path.write_text("\n".join(arr))
            written += 1

    print(f"[{split_dir.name}] YOLO labels written: {written} | images with 0 labels: {empty}")

# Run conversion on train/valid/test
for sub in ["train", "valid", "test"]:
    sd = Path(DATA_ROOT) / sub
    if sd.exists() and (sd / "images").exists():
        coco_to_yolo_convert(sd)


[train] YOLO labels written: 2095 | images with 0 labels: 0
[valid] YOLO labels written: 164 | images with 0 labels: 0
[test] YOLO labels written: 53 | images with 0 labels: 0


In [9]:
from pathlib import Path
data_yaml = Path(OUT_DIR)/"wheat_yolo_detection.yaml"
data_yaml.write_text(f"""# auto-generated
path: {DATA_ROOT}
train: train/images
val: valid/images
test: test/images
names:
  0: healthy
  1: unhealthy
""")
print("Wrote:", data_yaml)


Wrote: /mmfs1/home/jacks.local/kkumari/FHB_Project/runs_yolo_detection/wheat_yolo_detection.yaml


In [11]:
# --- Step 5: Build YOLO model without trying to download weights ---
from ultralytics import YOLO
import torch, os
from pathlib import Path

# If you have a pretrained .pt file already downloaded, put its path here
# Example Windows: r"F:\models\yolov8n.pt"
# Example Linux: "/home/you/models/yolov8n.pt"
WEIGHTS_LOCAL = None   # <-- change this to a local .pt file if available

device = 0 if torch.cuda.is_available() else "cpu"

def load_model_no_download():
    """Load YOLO model without internet fetch"""
    if WEIGHTS_LOCAL and Path(WEIGHTS_LOCAL).exists():
        print(f"Loading local weights: {WEIGHTS_LOCAL}")
        return YOLO(WEIGHTS_LOCAL)

    try:
        print("⚙️ Building from yolo11n.yaml (no download)...")
        return YOLO("yolo11n.yaml")   # smallest YOLOv11
    except Exception as e1:
        print("YOLO11 not available, falling back to yolov8n.yaml:", e1)
        return YOLO("yolov8n.yaml")   # fallback

# Build model
model = load_model_no_download()

# (Optional) print summary
try:
    model.info()
except Exception:
    pass

# --- Train the model ---
results = model.train(
    data=str(data_yaml),      # from Step 4
    imgsz=IMG_SIZE,
    epochs=EPOCHS,
    batch=BATCH,
    device=device,
    workers=0 if os.name == "nt" else 2,  # Windows requires workers=0
    project=str(OUT_DIR),
    name="train_yolo",
    pretrained=False,         # 🚨 prevent auto-download
    exist_ok=True
)

# --- Validate after training ---
metrics = model.val(
    data=str(data_yaml),
    imgsz=IMG_SIZE,
    device=device,
    project=str(OUT_DIR),
    name="val_yolo",
    workers=0 if os.name == "nt" else 2
)

print("Validation metrics:", metrics)


⚙️ Building from yolo11n.yaml (no download)...
YOLO11n summary: 181 layers, 2,624,080 parameters, 2,624,064 gradients, 6.6 GFLOPs
Ultralytics 8.3.204 🚀 Python-3.9.19 torch-2.2.2 CPU (Intel Xeon Gold 6342 CPU @ 2.80GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/mmfs1/home/jacks.local/kkumari/FHB_Project/runs_yolo_detection/wheat_yolo_detection.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=960, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.

/mmfs1/cm/shared/apps_local/python/3.11/envs/kayode/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 21.9±38.6 MB/s, size: 62.9 KB)
val: Scanning /mmfs1/home/jacks.local/kkumari/FHB_Project/Wheatproject-v1-coco/valid/labels... 164 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 164/164 906.7it/s 0.2s3s
val: New cache created: /mmfs1/home/jacks.local/kkumari/FHB_Project/Wheatproject-v1-coco/valid/labels.cache
Plotting labels to /mmfs1/home/jacks.local/kkumari/FHB_Project/runs_yolo_detection/train_yolo/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
Image sizes 960 train, 960 val


In [12]:
import pandas as pd
from pathlib import Path

# pick best weights saved by Ultralytics
best_dir = Path(OUT_DIR) / "train_yolo" / "weights"
best = None
if best_dir.exists():
    cands = sorted(best_dir.glob("best*.pt"))
    if cands:
        best = cands[-1]
assert best is not None, "best.pt not found; did training finish?"

detector = YOLO(str(best))
pred_dir = Path(OUT_DIR) / "pred_valid"

pred = detector.predict(
    source=str(Path(DATA_ROOT) / "valid" / "images"),
    imgsz=IMG_SIZE,
    conf=CONF_THR,
    device=device,
    save=True,         # saves images with drawn boxes
    save_txt=True,     # saves predictions as .txt
    project=str(OUT_DIR),
    name="pred_valid",
    exist_ok=True
)

rows = []
for r in pred:
    cls_ids = r.boxes.cls.int().tolist() if r.boxes is not None else []
    healthy   = sum(1 for c in cls_ids if c == 0)
    unhealthy = sum(1 for c in cls_ids if c == 1)
    total     = healthy + unhealthy
    ratio_unhealthy = (unhealthy / total) if total > 0 else 0.0
    rows.append({
        "image": Path(r.path).name,
        "count_healthy": healthy,
        "count_unhealthy": unhealthy,
        "total_spikes": total,
        "unhealthy_ratio": round(ratio_unhealthy, 4)
    })

df = pd.DataFrame(rows).sort_values("image").reset_index(drop=True)
csv_path = OUT_DIR / "counts_valid.csv"
df.to_csv(csv_path, index=False)
print("Saved counts CSV →", csv_path.as_posix())
df.head(10)



image 1/164 /mmfs1/home/jacks.local/kkumari/FHB_Project/Wheatproject-v1-coco/valid/images/IMG_0098_JPG.rf.3bd014f4c27d2371f33201c5a8d19750.jpg: 960x960 27 unhealthys, 121.0ms
image 2/164 /mmfs1/home/jacks.local/kkumari/FHB_Project/Wheatproject-v1-coco/valid/images/IMG_0268_JPG.rf.625e49bb15cbf96a8a0549c01a4fcdb3.jpg: 960x960 6 healthys, 13 unhealthys, 58.3ms
image 3/164 /mmfs1/home/jacks.local/kkumari/FHB_Project/Wheatproject-v1-coco/valid/images/IMG_0319_JPG.rf.eed2e0cb40460e93cdd30a268d8de6dc.jpg: 960x960 12 healthys, 17 unhealthys, 57.1ms
image 4/164 /mmfs1/home/jacks.local/kkumari/FHB_Project/Wheatproject-v1-coco/valid/images/IMG_0320_JPG.rf.39fbfd1d6afa69a0a057c25458bd765c.jpg: 960x960 9 healthys, 24 unhealthys, 56.2ms
image 5/164 /mmfs1/home/jacks.local/kkumari/FHB_Project/Wheatproject-v1-coco/valid/images/IMG_0336_JPG.rf.567d28ca026631bfe2f3585db6a395a4.jpg: 960x960 2 healthys, 13 unhealthys, 60.1ms
image 6/164 /mmfs1/home/jacks.local/kkumari/FHB_Project/Wheatproject-v1-coco/va

TypeError: unsupported operand type(s) for /: 'str' and 'str'

In [13]:
# Step 7: Inference on valid set + per-image counting CSV (robust)
import os
import pandas as pd
from pathlib import Path
from ultralytics import YOLO
import torch

device = 0 if torch.cuda.is_available() else "cpu"

best_dir = Path(OUT_DIR) / "train_yolo" / "weights"
assert best_dir.exists(), f"Weights folder not found: {best_dir}"

# Try best*.pt, else last.pt
cands = sorted(best_dir.glob("best*.pt"))
if cands:
    best = cands[-1]
else:
    last = best_dir / "last.pt"
    assert last.exists(), "No best.pt or last.pt found. Did Step 5 finish?"
    best = last

print("Using weights:", best)
detector = YOLO(str(best))

pred = detector.predict(
    source=str(Path(DATA_ROOT) / "valid" / "images"),
    imgsz=IMG_SIZE,
    conf=CONF_THR,
    device=device,
    save=True,          # save images with boxes
    save_txt=True,      # save txt predictions
    project=str(OUT_DIR),
    name="pred_valid",
    exist_ok=True,
    workers=0 if os.name == "nt" else 2
)

rows = []
for r in pred:
    cls_ids = r.boxes.cls.int().tolist() if r.boxes is not None else []
    healthy   = sum(1 for c in cls_ids if c == 0)
    unhealthy = sum(1 for c in cls_ids if c == 1)
    total     = healthy + unhealthy
    ratio_unhealthy = (unhealthy / total) if total > 0 else 0.0
    rows.append({
        "image": Path(r.path).name,
        "count_healthy": healthy,
        "count_unhealthy": unhealthy,
        "total_spikes": total,
        "unhealthy_ratio": round(ratio_unhealthy, 4)
    })

df = pd.DataFrame(rows).sort_values("image").reset_index(drop=True)
csv_path = Path(OUT_DIR) / "counts_valid.csv"
df.to_csv(csv_path, index=False)
print("Saved counts CSV →", csv_path)
df.head(10)


Using weights: /mmfs1/home/jacks.local/kkumari/FHB_Project/runs_yolo_detection/train_yolo/weights/best.pt

image 1/164 /mmfs1/home/jacks.local/kkumari/FHB_Project/Wheatproject-v1-coco/valid/images/IMG_0098_JPG.rf.3bd014f4c27d2371f33201c5a8d19750.jpg: 960x960 27 unhealthys, 60.5ms
image 2/164 /mmfs1/home/jacks.local/kkumari/FHB_Project/Wheatproject-v1-coco/valid/images/IMG_0268_JPG.rf.625e49bb15cbf96a8a0549c01a4fcdb3.jpg: 960x960 6 healthys, 13 unhealthys, 59.4ms
image 3/164 /mmfs1/home/jacks.local/kkumari/FHB_Project/Wheatproject-v1-coco/valid/images/IMG_0319_JPG.rf.eed2e0cb40460e93cdd30a268d8de6dc.jpg: 960x960 12 healthys, 17 unhealthys, 56.3ms
image 4/164 /mmfs1/home/jacks.local/kkumari/FHB_Project/Wheatproject-v1-coco/valid/images/IMG_0320_JPG.rf.39fbfd1d6afa69a0a057c25458bd765c.jpg: 960x960 9 healthys, 24 unhealthys, 57.9ms
image 5/164 /mmfs1/home/jacks.local/kkumari/FHB_Project/Wheatproject-v1-coco/valid/images/IMG_0336_JPG.rf.567d28ca026631bfe2f3585db6a395a4.jpg: 960x960 2 health

,image,count_healthy,count_unhealthy,total_spikes,unhealthy_ratio
0,IMG_0098_JPG.rf.3bd014f4c27d2371f33201c5a8d197...,0,27,27,1.0000
1,IMG_0268_JPG.rf.625e49bb15cbf96a8a0549c01a4fcd...,6,13,19,0.6842
2,IMG_0319_JPG.rf.eed2e0cb40460e93cdd30a268d8de6...,12,17,29,0.5862
3,IMG_0320_JPG.rf.39fbfd1d6afa69a0a057c25458bd76...,9,24,33,0.7273
4,IMG_0336_JPG.rf.567d28ca026631bfe2f3585db6a395...,2,13,15,0.8667
5,IMG_0349_JPG.rf.20405c28ae8c5e093b0b72fe1c3c6a...,4,11,15,0.7333
6,IMG_0390_JPG.rf.fa9a8e58e65234498cdec5b3b9a258...,0,11,11,1.0000
7,IMG_0428_JPG.rf.fe13d4376df6c9a31b226461726998...,17,21,38,0.5526
8,IMG_0479_JPG.rf.7db57d1291a5444a68462be78e2b04...,8,26,34,0.7647
9,IMG_0536_JPG.rf.13b1b04faf4120a6593f9a542b7bdf...,5,6,11,0.5455
